In [140]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import datetime
from geographiclib.geodesic import Geodesic
plt.rcParams['figure.max_open_warning'] = 0

In [141]:
#加载训练数据
train_dataset = pd.read_csv("temp5.csv")

In [142]:
train_dataset = train_dataset[~train_dataset.id.isin([756244])]

In [143]:
train_dataset.reset_index(inplace = True)

In [144]:
move_u = []
move_v = []
for i in range(train_dataset.shape[0]-30):
    if train_dataset.id[i] == train_dataset.id[i+30]:
        distance = Geodesic.WGS84.Inverse(train_dataset.lat[i],train_dataset.lng[i],train_dataset.lat[i+30],train_dataset.lng[i+30])
        move_u.append(distance["s12"] * math.sin(math.radians(distance["azi1"]))) #move_u: 位移东向向量
        move_v.append(distance["s12"] * math.cos(math.radians(distance["azi1"]))) #move_v: 位移北向向量
    else:
        move_u.append(np.nan)
        move_v.append(np.nan)
for i in range(30):
    move_u.append(np.nan)
    move_v.append(np.nan)    
train_dataset["move_u_30"] = move_u
train_dataset["move_v_30"] = move_v

In [145]:
train_dataset.dropna(inplace=True)
#train_u = train_dataset[["wd_u_avg","Oflow_u_avg"]]
#train_v = train_dataset[["wd_v_avg","Oflow_v_avg"]]
train_u = train_dataset[["lng","lat","wd_u_avg","Oflow_u_avg","wd_v_avg","Oflow_v_avg"]]
train_v = train_dataset[["lng","lat","wd_u_avg","Oflow_u_avg","wd_v_avg","Oflow_v_avg"]]
y_u  = train_dataset["move_u_30"]
y_v  = train_dataset["move_v_30"]

In [146]:
from sklearn import linear_model
# clf_u = linear_model.LinearRegression(normalize=True)
# clf_v = linear_model.LinearRegression(normalize=True)
#clf_u = linear_model.RANSACRegressor()
#clf_v = linear_model.RANSACRegressor()
#clf_u.fit(train_u[["wd_u_avg","Oflow_u_avg"]][0:40000],y_u[0:40000])
#clf_v.fit(train_v[["wd_v_avg","Oflow_v_avg"]][0:40000],y_v[0:40000])


from sklearn import ensemble
clf_u = ensemble.GradientBoostingRegressor(n_estimators=20)#这里使用100个决策树
clf_v = ensemble.GradientBoostingRegressor(n_estimators=20)#这里使用100个决策树

# clf_u = ensemble.RandomForestRegressor(n_estimators=100)#这里使用100个决策树
# clf_v = ensemble.RandomForestRegressor(n_estimators=100)#这里使用100个决策树

from sklearn.tree import ExtraTreeRegressor
#clf_u = ExtraTreeRegressor()#这里使用100个决策树
#clf_v = ExtraTreeRegressor()#这里使用100个决策树

from sklearn import neighbors
#clf_u = neighbors.KNeighborsRegressor()
#clf_v = neighbors.KNeighborsRegressor()
#clf_u.fit(train_u[["wd_u_avg","Oflow_u_avg"]][0:40000],y_u[0:40000])
#clf_v.fit(train_v[["wd_v_avg","Oflow_v_avg"]][0:40000],y_v[0:40000])

from sklearn import svm
# clf_u = svm.SVR(kernel='linear')
# clf_v = svm.SVR(kernel='linear')

from sklearn.ensemble import BaggingRegressor
#clf_u = BaggingRegressor()
#clf_v = BaggingRegressor()

from sklearn import ensemble
#clf_u = ensemble.AdaBoostRegressor(n_estimators=10)#这里使用50个决策树
#clf_v = ensemble.AdaBoostRegressor(n_estimators=10)#这里使用50个决策树

from sklearn import ensemble
#clf_u = BaggingRegressor()
#clf_v = BaggingRegressor()



from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet

from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor,ExtraTreesRegressor


from xgboost import XGBRegressor

from lightgbm import LGBMRegressor
# 支持向量机
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler,PolynomialFeatures

#KNeighborsRegressor()  0.9865357257293653  AdaBoostRegressor()  0.6567539848630852   LGBMRegressor()  0.95  XGBRegressor()  0.9892752102085149
# clf_u = GradientBoostingRegressor(n_estimators=60)
# clf_v = GradientBoostingRegressor(n_estimators=60)

clf_u.fit(train_u,y_u)
clf_v.fit(train_v,y_v)

GradientBoostingRegressor(n_estimators=20)

In [147]:
clf_u.score(train_u,y_u)

0.5040559433913973

In [148]:
clf_v.score(train_v,y_v)

0.663665625788435

In [149]:
#加载测试数据
test_dataset = pd.read_csv("val.csv",parse_dates = [["date","time"]])

In [150]:
test_dataset

,date_time,id,lng,lat
0,2019-12-31 04:02:00,808199,114.037879,19.232220
1,2020-01-08 04:02:00,102006,111.843168,18.531674


In [151]:
def GPS_filter(data,lon,lat,num):
    dis_step = data["lat"].diff().abs().mean().mean()
    distance = 0.5*dis_step*(num**0.5)
    GPSfilter = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
    while GPSfilter.sum().sum()!=num:
        if GPSfilter.sum().sum()>num:
            max_dis = [data["lat"][GPSfilter].max().max()-lat,
                        data["lon"][GPSfilter].max().max()-lon,
                        lat-data["lat"][GPSfilter].min().min(),
                        lon-data["lon"][GPSfilter].min().min()]
            distance = max(max_dis)
            GPSfilter_range = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
            GPSfilter_range = GPSfilter ^ GPSfilter_range
            GPSfilter_range_num = len(np.where(GPSfilter_range)[0])          
            if GPSfilter_range_num>1 and (GPSfilter.sum().sum()-num)<GPSfilter_range_num:
                range_num = num+GPSfilter_range_num-GPSfilter.sum().sum()
                GPSfilter = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
                coord = np.where(GPSfilter_range)
                #print(range_num)  
                for i in range(range_num):
                    GPSfilter.iloc[coord[0][i], coord[1][i]] = True
            else:
                    GPSfilter = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
        elif GPSfilter.sum().sum()<num:
            distance = distance +dis_step*0.0999
            GPSfilter = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
        #print(GPSfilter.sum().sum())
    return GPSfilter

In [152]:
test_pred = pd.DataFrame(columns=['date_time', 'id', 'lng', 'lat','wd_u_avg','wd_v_avg','Oflow_u_avg','Oflow_v_avg','move_u','move_v'])

In [153]:
test_pred

,date_time,id,lng,lat,wd_u_avg,wd_v_avg,Oflow_u_avg,Oflow_v_avg,move_u,move_v


In [154]:
#加载测试数据
id_is_on_land = pd.read_csv("id_is_on_land.csv")

In [155]:
#筛选数据集
test_dataset = test_dataset[test_dataset.id.isin(id_is_on_land[id_is_on_land.near_land==False]["Unnamed: 0"])]
#test_dataset = test_dataset[test_dataset.id.isin(id_is_on_land[id_is_on_land.is_on_land==False]["Unnamed: 0"])]

In [156]:
test_dataset

,date_time,id,lng,lat
0,2019-12-31 04:02:00,808199,114.037879,19.232220
1,2020-01-08 04:02:00,102006,111.843168,18.531674


In [157]:
from tqdm import tqdm
for index, rowdata in tqdm(test_dataset.iterrows()): #test2_dataset.iterrows():
    row=rowdata.copy()
    print(rowdata)
    for ftimes in tqdm(range(96)):
        temp_u = []
        temp_v = []    
        date_temp = row.date_time + datetime.timedelta(hours=-20) #风场
        hour_temp = date_temp.hour
        if date_temp.strftime("%Y%m%d") not in ["20191221","20200319","20200330"]:
            wd=Dataset('wind/wind_hour_%s12.nc' % date_temp.strftime("%Y%m%d"))
        else:
            date_temp = date_temp + datetime.timedelta(hours=-24)
            hour_temp = date_temp.hour + 24
            wd=Dataset('wind/wind_hour_%s12.nc' % date_temp.strftime("%Y%m%d"))
        wd_lon = wd.variables['LONGITUDE1_151'][:].data
        wd_lat = wd.variables['LATITUDE1_151'][:].data
        wd_u = wd.variables['U10'][:].data[hour_temp]
        wd_v = wd.variables['V10'][:].data[hour_temp]
        wd_db = pd.DataFrame(np.zeros(((wd_u.shape[0], wd_u.shape[1]*4))),index=list(range(wd_u.shape[0])),columns=[["lat"]*wd_u.shape[1]+["lon"]*wd_u.shape[1]+["u"]*wd_u.shape[1]+["v"]*wd_u.shape[1],list(range(wd_u.shape[1]))*4])
        wd_db["lat"] = wd_lat
        wd_db["lon"] = wd_lon
        wd_db["lon"] = wd_db["lon"].T
        wd_db["u"] = wd_u
        wd_db["v"] = wd_v
        GPS_sub4 = GPS_filter(wd_db,row.lng,row.lat,4)
        dis_weights = []
        for i in zip(wd_db["lon"][GPS_sub4].unstack().dropna(how='all'),wd_db["lat"][GPS_sub4].unstack().dropna(how='all')):
            dis_weights.append(1/Geodesic.WGS84.Inverse(row.lat,row.lng,i[1], i[0])["s12"])
        u1 = np.average(wd_db["u"][GPS_sub4].unstack().dropna(how='all'),weights = dis_weights)
        v1 = np.average(wd_db["v"][GPS_sub4].unstack().dropna(how='all'),weights = dis_weights)
        
        date_temp = row.date_time + datetime.timedelta(hours=-19) #风场
        hour_temp = date_temp.hour
        min_temp = date_temp.minute
        wd_lon = wd.variables['LONGITUDE1_151'][:].data
        wd_lat = wd.variables['LATITUDE1_151'][:].data
        wd_u = wd.variables['U10'][:].data[hour_temp]
        wd_v = wd.variables['V10'][:].data[hour_temp]
        wd_db = pd.DataFrame(np.zeros(((wd_u.shape[0], wd_u.shape[1]*4))),index=list(range(wd_u.shape[0])),columns=[["lat"]*wd_u.shape[1]+["lon"]*wd_u.shape[1]+["u"]*wd_u.shape[1]+["v"]*wd_u.shape[1],list(range(wd_u.shape[1]))*4])
        wd_db["lat"] = wd_lat
        wd_db["lon"] = wd_lon
        wd_db["lon"] = wd_db["lon"].T
        wd_db["u"] = wd_u
        wd_db["v"] = wd_v
        #GPS_sub4 = GPS_filter(wd_db,row.lng,row.lat,4)
        #dis_weights = []
        #for i in zip(wd_db["lon"][GPS_sub4].unstack().dropna(how='all'),wd_db["lat"][GPS_sub4].unstack().dropna(how='all')):
        #    dis_weights.append(1/Geodesic.WGS84.Inverse(row.lat,row.lng,i[1], i[0])["s12"])
        u2 = np.average(wd_db["u"][GPS_sub4].unstack().dropna(how='all'),weights = dis_weights)
        v2 = np.average(wd_db["v"][GPS_sub4].unstack().dropna(how='all'),weights = dis_weights)       
        
        temp_u.append(np.average([u1,u2],weights = [(60-min_temp),min_temp]))
        temp_v.append(np.average([v1,v2],weights = [(60-min_temp),min_temp]))

        date_temp = row.date_time + datetime.timedelta(hours=-8) #洋流
        hour_temp = date_temp.hour
        Oflow=Dataset('current/current_hour_%s.nc' % date_temp.strftime("%Y%m%d"))
        Oflow_lon = Oflow.variables['lon'][:].data.T  # 经度间隔0.005度
        Oflow_lat = Oflow.variables['lat'][:].data.T   # 纬度间隔0.01度
        Oflow_u = Oflow.variables['u'][:].data[hour_temp][0].T
        Oflow_v = Oflow.variables['v'][:].data[hour_temp][0].T
        Oflow_db = pd.DataFrame(np.zeros(((Oflow_u.shape[0], Oflow_u.shape[1]*4))),index=list(range(Oflow_u.shape[0])),columns=[["lat"]*Oflow_u.shape[1]+["lon"]*Oflow_u.shape[1]+["u"]*Oflow_u.shape[1]+["v"]*Oflow_u.shape[1],list(range(Oflow_u.shape[1]))*4])
        Oflow_db["lat"] = Oflow_lat
        Oflow_db["lon"] = Oflow_lon
        Oflow_db["u"] = Oflow_u
        Oflow_db["v"] = Oflow_v
        GPS_sub9 = GPS_filter(Oflow_db,row.lng,row.lat,4)
        dis_weights = []
        for j in zip(Oflow_db["lon"][GPS_sub9].unstack().dropna(how='all'),Oflow_db["lat"][GPS_sub9].unstack().dropna(how='all')):
            dis_weights.append(1/Geodesic.WGS84.Inverse(row.lat,row.lng,j[1], j[0])["s12"])
        u3 = np.average(Oflow_db["u"][GPS_sub9].unstack().dropna(how='all'),weights = dis_weights)
        v3 = np.average(Oflow_db["v"][GPS_sub9].unstack().dropna(how='all'),weights = dis_weights)
        u3 = u3 if -10<u3<10 else 0
        v3 = v3 if -10<v3<10 else 0
        #temp_u.append(tempu)
        #temp_v.append(tempv)
        
        date_temp = row.date_time + datetime.timedelta(hours=-7) #洋流
        hour_temp = date_temp.hour
        min_temp = date_temp.minute
        Oflow=Dataset('current/current_hour_%s.nc' % date_temp.strftime("%Y%m%d"))
        Oflow_lon = Oflow.variables['lon'][:].data.T  # 经度间隔0.005度
        Oflow_lat = Oflow.variables['lat'][:].data.T   # 纬度间隔0.01度
        Oflow_u = Oflow.variables['u'][:].data[hour_temp][0].T
        Oflow_v = Oflow.variables['v'][:].data[hour_temp][0].T
        Oflow_db = pd.DataFrame(np.zeros(((Oflow_u.shape[0], Oflow_u.shape[1]*4))),index=list(range(Oflow_u.shape[0])),columns=[["lat"]*Oflow_u.shape[1]+["lon"]*Oflow_u.shape[1]+["u"]*Oflow_u.shape[1]+["v"]*Oflow_u.shape[1],list(range(Oflow_u.shape[1]))*4])
        Oflow_db["lat"] = Oflow_lat
        Oflow_db["lon"] = Oflow_lon
        Oflow_db["u"] = Oflow_u
        Oflow_db["v"] = Oflow_v
        #GPS_sub9 = GPS_filter(Oflow_db,row.lng,row.lat,9)
        #dis_weights = []
        #for j in zip(Oflow_db["lon"][GPS_sub9].unstack().dropna(how='all'),Oflow_db["lat"][GPS_sub9].unstack().dropna(how='all')):
        #    dis_weights.append(1/Geodesic.WGS84.Inverse(row.lat,row.lng,j[1], j[0])["s12"])
        u4 = np.average(Oflow_db["u"][GPS_sub9].unstack().dropna(how='all'),weights = dis_weights)
        v4 = np.average(Oflow_db["v"][GPS_sub9].unstack().dropna(how='all'),weights = dis_weights)
        u4 = u4 if -10<u4<10 else 0
        v4 = v4 if -10<v4<10 else 0
        temp_u.append(np.average([u3,u3],weights = [(60-min_temp),min_temp]))
        temp_v.append(np.average([v4,v4],weights = [(60-min_temp),min_temp]))
        
        train_temp = [row.lng,row.lat,temp_u[0],temp_u[1],temp_v[0],temp_v[1]]

        distance_u = clf_u.predict([train_temp]) #distance_u: 位移东向向量
        distance_v = clf_v.predict([train_temp]) #distance_v: 位移北向向量
        distance_temp = (distance_u**2+distance_v**2)**0.5*(distance_v/abs(distance_v))*0.82 #0.8倍距离，消除累计偏差
        
        new = [row.date_time,row.id,row.lng,row.lat,temp_u[0],temp_v[0],temp_u[1],temp_v[1],distance_u[0],distance_v[0]]
        new_df=pd.DataFrame([new],columns=['date_time', 'id', 'lng', 'lat','wd_u_avg','wd_v_avg','Oflow_u_avg','Oflow_v_avg','move_u','move_v'])
        test_pred = test_pred.append(new_df,ignore_index=True)

        row.date_time = row.date_time + datetime.timedelta(minutes=30)
        row_temp = Geodesic.WGS84.Direct(row.lat, row.lng, math.degrees(math.atan(distance_u/distance_v)), distance_temp)

        row.lat = row_temp["lat2"]
        row.lng = row_temp["lon2"]   

0it [00:00, ?it/s]
  0%|                                                                                           | 0/96 [00:00<?, ?it/s]

date_time    2019-12-31 04:02:00
id                        808199
lng                   114.037879
lat                     19.23222
Name: 0, dtype: object


C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(

  1%|▊                                                                                  | 1/96 [00:00<00:29,  3.20it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(

  2%|█▋                                    

 17%|█████████████▋                                                                    | 16/96 [00:05<00:25,  3.16it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(

 18%|██████████████▌                                                                   | 17/96 [00:05<00:25,  3.09it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, bu

 32%|██████████████████████████▍                                                       | 31/96 [00:09<00:20,  3.24it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(

 33%|███████████████████████████▎                                                      | 32/96 [00:10<00:19,  3.24it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, bu

...

 32%|██████████████████████████▍                                                       | 31/96 [00:11<00:23,  2.71it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(

 33%|███████████████████████████▎                                                      | 32/96 [00:11<00:24,  2.60it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, bu

 48%|███████████████████████████████████████▎                                          | 46/96 [00:17<00:18,  2.65it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(

 49%|████████████████████████████████████████▏                                         | 47/96 [00:17<00:18,  2.70it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, bu

 64%|████████████████████████████████████████████████████                              | 61/96 [00:22<00:13,  2.68it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(

 65%|████████████████████████████████████████████████████▉                             | 62/96 [00:23<00:12,  2.71it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, bu

 79%|████████████████████████████████████████████████████████████████▉                 | 76/96 [00:28<00:07,  2.51it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(

 80%|█████████████████████████████████████████████████████████████████▊                | 77/96 [00:28<00:07,  2.55it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, bu

 95%|█████████████████████████████████████████████████████████████████████████████▋    | 91/96 [00:34<00:01,  2.66it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 92/96 [00:34<00:01,  2.68it/s]C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
C:\Users\10521\AppData\Roaming\Python\Python38\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, bu

In [158]:
test_pred.to_csv("GBDT_20.csv",index=False)

In [159]:
test_pred

,date_time,id,lng,lat,wd_u_avg,wd_v_avg,Oflow_u_avg,Oflow_v_avg,move_u,move_v
0,2019-12-31 04:02:00,808199,114.037879,19.232220,-8.042372,-5.547577,0.003530,-0.242695,-775.992334,-412.484621
1,2019-12-31 04:32:00,808199,114.031827,19.229164,-8.207223,-5.610332,0.004747,-0.243969,-775.992334,-412.484621
2,2019-12-31 05:02:00,808199,114.025776,19.226109,-8.362338,-5.680679,-0.032501,-0.299955,-812.205613,-486.098675
3,2019-12-31 05:32:00,808199,114.019442,19.222508,-8.361619,-5.869433,-0.032089,-0.300801,-929.701691,-486.098675
4,2019-12-31 06:02:00,808199,114.012192,19.218907,-8.368739,-6.056623,-0.085963,-0.341078,-929.701691,-486.098675
...,...,...,...,...,...,...,...,...,...,...
187,2020-01-10 01:32:00,102006,111.292617,18.132935,-6.859055,-3.092633,0.062655,-0.054335,-669.112519,-457.428637
188,2020-01-10 02:02:00,102006,111.287432,18.129546,-7.135215,-2.884547,0.062450,-0.059165,-669.112519,-457.428637
189,2020-01-10 02:32:00,102006,111.282248,18.126157,-7.204316,-2.741161,0.062675,-0.058751,-669.112519,-417.779968
190,2020-01-10 03:02:00,102006,111.277063,18.123061,-7.260097,-2.608488,0.059319,-0.060785,-669.112519,-417.779968


In [164]:
submit_dataset = pd.read_csv("n_estimators=60.csv")
submit_dataset["date"] = test_pred.date_time.dt.strftime("%Y/%m/%d")
submit_dataset["time"] = test_pred.date_time.dt.strftime("%H:%M")

In [165]:
submit_dataset

,date_time,id,lng,lat,wd_u_avg,wd_v_avg,Oflow_u_avg,Oflow_v_avg,move_u,move_v,date,time
0,2019-12-31 04:02:00,808199,114.037879,19.232220,-8.042372,-5.547577,0.003530,-0.242695,-848.313726,-67.863958,2019/12/31,04:02
1,2019-12-31 04:32:00,808199,114.031263,19.231717,-8.208291,-5.610895,0.017454,-0.252836,-772.131109,-67.863958,2019/12/31,04:32
2,2019-12-31 05:02:00,808199,114.025241,19.231215,-8.363446,-5.682478,-0.033506,-0.300336,-984.133611,-163.577560,2019/12/31,05:02
3,2019-12-31 05:32:00,808199,114.017567,19.230003,-8.353897,-5.870874,-0.033522,-0.301669,-984.133611,-163.577560,2019/12/31,05:32
4,2019-12-31 06:02:00,808199,114.009892,19.228791,-8.341099,-6.057592,-0.089074,-0.341031,-976.649883,-133.990695,2019/12/31,06:02
...,...,...,...,...,...,...,...,...,...,...,...,...
187,2020-01-10 01:32:00,102006,111.296551,18.335295,-7.144495,-2.717472,0.104401,-0.031476,-621.779101,-119.226381,2020/01/10,01:32
188,2020-01-10 02:02:00,102006,111.291727,18.334411,-7.274877,-2.652741,0.104607,-0.023488,-621.779101,-46.521977,2020/01/10,02:02
189,2020-01-10 02:32:00,102006,111.286904,18.334067,-7.336051,-2.482471,0.105076,-0.022804,-651.596988,-46.521977,2020/01/10,02:32
190,2020-01-10 03:02:00,102006,111.281849,18.333722,-7.372788,-2.330774,0.100475,-0.022531,-730.408789,-46.521977,2020/01/10,03:02


In [166]:
#转换格式
submit_dataset = submit_dataset[['id','date','time','lng','lat']]

In [167]:
submit_dataset.to_csv("GBDT_60_pred.csv")

In [57]:
#加载靠岸数据
id_is_on_land = pd.read_csv("id_is_on_land.csv")
test_dataset = pd.read_csv("test_data.csv",parse_dates = [["date","time"]])
sub_order = test_dataset.id.tolist()
test_dataset = test_dataset[test_dataset.id.isin(id_is_on_land[id_is_on_land.near_land==True]["Unnamed: 0"])] 
#test_dataset = test_dataset[test_dataset.id.isin(id_is_on_land[id_is_on_land.is_on_land==True]["Unnamed: 0"])]
test_dataset["date"] = test_dataset.date_time.dt.strftime("%Y/%m/%d")
test_dataset["time"] = test_dataset.date_time.dt.strftime("%H:%M")
#生成靠岸数据
submit0 = pd.DataFrame(columns=['id','date','time','lng','lat'])
from tqdm import tqdm
for i in tqdm(range(test_dataset.shape[0])):
    new = test_dataset[i:i+1].copy()
    for j in range(96):
        new["date"] = new.date_time.dt.strftime("%Y/%m/%d")
        new["time"] = new.date_time.dt.strftime("%H:%M")
        new.date_time = new.date_time + datetime.timedelta(minutes=30)
        submit0 = submit0.append(new[['id','date','time','lng','lat']],ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:04<00:00,  4.74it/s]


In [ ]:
# submit0.to_csv("AdaBoostRegressor.csv",index=False)

In [58]:
submit_dataset = pd.concat([submit0,submit_dataset],axis=0)

In [59]:
#排序
submit_dataset = submit_dataset.reset_index()
submit_dataset['id'] = submit_dataset['id'].astype('category')
submit_dataset['id'].cat.set_categories(sub_order, inplace=True) # inplace = True，使 set_categories生效
submit_dataset.sort_values(['id','index'], ascending=[True,True],inplace=True)
submit_dataset = submit_dataset.drop("index",axis = 1)

In [60]:
submit_dataset.to_csv("n_estimators=60_Result.csv",index=False)